# TDT17: Key concept (FCNNs): Regularization: Heuristics: Early stopping, Data augmentation, Noise

This notebook contains a Given-Find task for the key concept of regularization in deep learning. The task revolves around calculating updated bounding boxes for augmented images. Computer vision data augmentation is not yet very well supported in a lot of libraries, and was only very recently [added to PyTorch](https://pytorch.org/blog/extending-torchvisions-transforms-to-object-detection-segmentation-and-video-tasks/) in the transforms V2 API. The task is therefore actually something you may see yourself having to implement, which makes it very useful.

## Prerequisites

This notebook requires the following Python packages:
 - requests (optional, for downloading the image)
 - matplotlib

*nix commands for setting up environment and installing packages;
```bash
python3 -m venv venv
source venv/bin/activate
pip install requests numpy matplotlib
```

## Task description

This task assumes familiarity with the topics at hand and does not go into depth about the theory behind them. We are going to be using the below image of a parking lot, with the bounding boxes of the cars in the image, and calculating the updated bounding boxes for the augmented images. The image is augmented using a few different transforms. The code to apply the transform to the image is provided, and the task is to calculate the updated bounding boxes for the augmented images.

![Parking lot with some cars](parking_lot.png)

*If you do not see this image run the code below to download it*

In [ ]:
import requests

img_data = requests.get("https://raw.githubusercontent.com/LorgeN/TDT17/master/parking_lot.png").content
with open('parking_lot.png', 'wb') as handler:
    handler.write(img_data)

The methods below allow us to visualise the bounding boxes on the image, and may be useful later to verify your findings

In [ ]:
from typing import List
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image


class BB:
    x: int  # x coordinate of the top left corner pixel
    y: int  # y coordinate of the top left corner pixel
    w: int  # width of the bounding box (in pixels)
    h: int  # height of the bounding box (in pixels)

    def __init__(self, x: int, y: int, w: int, h: int):
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def with_x(self, x: int):
        return BB(x, self.y, self.w, self.h)

    def with_y(self, y: int):
        return BB(self.x, y, self.w, self.h)

    def with_xy(self, x: int, y: int):
        return BB(x, y, self.w, self.h)

    def with_wh(self, w: int, h: int):
        return BB(self.x, self.y, w, h)

    def __str__(self):
        return f"BB(x={self.x}, y={self.y}, w={self.w}, h={self.h})"

    def __repr__(self):
        return self.__str__()


def draw_boxes(boxes: List[BB], im: Image):
    fig, ax = plt.subplots()
    ax.imshow(im)

    for box in boxes:
        rect = patches.Rectangle(
            (box.x, box.y), box.w, box.h, linewidth=1, edgecolor="r", facecolor="none"
        )
        ax.add_patch(rect)

    plt.show()

Below we define the bounding boxes for the default image

In [ ]:
IMAGE = Image.open("parking_lot.png")
DEFAULT_BOUNDING_BOXES = [
    BB(100, 220, 60, 75),
    BB(180, 270, 50, 85),
    BB(270, 190, 40, 85),
    BB(395, 280, 40, 90),
    BB(440, 195, 40, 85),
    BB(480, 280, 40, 85),
]

draw_boxes(DEFAULT_BOUNDING_BOXES, IMAGE)

## Task 1: Resized image

The code below resizes the image. This is a very useful transform since we in many cases may not have a uniform image size in our data, and some other transforms change the image size, so applying this transform as the last in your "chain" is a good way to ensure uniform data. The task is to calculate the updated bounding boxes in the skeleton code.

We are essentially "multiplying" the image by some factors in the x and y direction, and the bounding boxes should be updated accordingly. These factors are equal to

$$S_x = \frac{W_{new}}{W_{old}}$$

$$S_y = \frac{H_{new}}{H_{old}}$$

where $W$ and $H$ are the width and height of the image, respectively.

With these factors we can calculate the updated bounding boxes as

$$x_{new} = x_{old} \cdot S_x$$

$$y_{new} = y_{old} \cdot S_y$$

$$w_{new} = w_{old} \cdot S_x$$

$$h_{new} = h_{old} \cdot S_y$$

where $x$ and $y$ are the coordinates of the top left corner of the bounding box, and $w$ and $h$ are the width and height of the bounding box, respectively.

In [ ]:
original_dimensions = IMAGE.size
resized_image = IMAGE.resize((600, 400))

resized_bounding_boxes = [
    BB(
        box.x, # TODO: Update x coordinate
        box.y, # TODO: Update y coordinate
        box.w, # TODO: Update w
        box.h, # TODO: Update h
    )
    for box in DEFAULT_BOUNDING_BOXES
]

draw_boxes(resized_bounding_boxes, resized_image)

## Task 2: Flipped image

The code below flips the image horizontally. The task is to implement the logic for calculating the updated bounding boxes for the flipped image in the skeleton code below. 

The math for this transform is relatively straightforward. Our width and heigh remains the same, and since we are flipping horizontally, we only need to update our x coordinates. We have to remember that our reference point for the coordinate is the top left corner of the box. This means that the new x coordinate is equal to $$x_{new} = \text{image width} - \text{box width} - x_{old}$$

In [ ]:
flipped_image = IMAGE.transpose(Image.FLIP_LEFT_RIGHT)

flipped_bounding_boxes = [
    box.with_x(box.x) for box in DEFAULT_BOUNDING_BOXES  # TODO: Update x coordinate
]

draw_boxes(flipped_bounding_boxes, flipped_image)

# Task 3: Cropped image

The code below crops the image. The task is to implement the logic for calculating the updated bounding boxes for the cropped image in the skeleton code below.

This does not involve scaling, making it a question of translating the bounding boxes. Since we are cropping the image, we need to make sure that the bounding boxes are still within the image. This means that we need to check if the bounding box is outside the image, and if it is, we should either remove it or update the size.

First we should translate the boxes by the top left crop, which in this case is both 60 pixels. We do this simply by subtracting 60 from the x and y coordinates:

$$x_{new} = x_{old} - 60$$

$$y_{new} = y_{old} - 60$$

Then we need to check if the bounding box is outside the image. This has to be done as several steps:

1. If $x < 0$ -> $x = 0$ and update width by $w = w + x$ since we are removing $x$ pixels from the left side of the image
2. If $y < 0$ -> $y = 0$ and update height by $h = h + y$ since we are removing $y$ pixels from the top of the image
3. If $x + w > W$ -> $w = W - x$ since we are removing $x$ pixels from the right side of the image
4. If $y + h > H$ -> $h = H - y$ since we are removing $y$ pixels from the bottom of the image

where $W$ and $H$ are the width and height of the image, respectively.

Now, if the bounding box is outside the image, we should remove it. We can check this by checking if the width or height is less than or equal to 0. If it is, we should remove the bounding box.

In [ ]:
cropped_image = IMAGE.crop((60, 60, 440, 340))

cropped_bounding_boxes = []

for box in DEFAULT_BOUNDING_BOXES:
    updated = box  # TODO: Update box

    cropped_bounding_boxes.append(updated)

draw_boxes(cropped_bounding_boxes, cropped_image)

# Task 4: Rotate image

The code below rotates the image 3 degrees around the center point. The task is to implement the logic for calculating the updated bounding boxes for the rotated image in the skeleton code below.

The math for this transform is a bit more involved. We should consider the top left and bottom right corner of the bounding box, and rotate it around the center point of the image. We can calculate the center point of the image as

$$x_{center} = \frac{W}{2}$$

$$y_{center} = \frac{H}{2}$$

where $W$ and $H$ are the width and height of the image, respectively.

We can then use 

$$x_{new} = x_{center} + (x_{old} - x_{center}) \cdot \cos(\theta) - (y_{old} - y_{center}) \cdot \sin(\theta)$$

$$y_{new} = y_{center} + (x_{old} - x_{center}) \cdot \sin(\theta) + (y_{old} - y_{center}) \cdot \cos(\theta)$$

to rotate counter-clockwise where $\theta$ is the angle of rotation, which in this case is 3 degrees. Since this rotation is counter-clockwise we will need to use -3 degrees instead.

Since we can not represent an angled bounding box, we need to calculate the new width and height of the bounding box. We can do this by calculating the distance between the corners of the bounding box.

We can find the bottom right corner using

$$x_{bottom right} = x_{old} + w_{old}$$

$$y_{bottom right} = y_{old} + h_{old}$$

**Hint: Remember that python math uses radians! Use `math.radians` to convert from degrees to radians**

In [ ]:
import math

rotated_image = IMAGE.rotate(3)

rotated_bounding_boxes = []


for box in DEFAULT_BOUNDING_BOXES:
    updated = box # TODO: Update box coordinates

    rotated_bounding_boxes.append(updated)


draw_boxes(rotated_bounding_boxes, rotated_image)